# Imports

In [66]:
print('Loading libraries')
start_time <- Sys.time()
suppressPackageStartupMessages({
    library(data.table)
    library(ggplot2)
    library(dplyr)
    library(stringr)
    library(rstatix)
    library(tidyr)
    library(phenoptr)
    }
)
print('Library load finished')
print(Sys.time()-start_time)


hpc =  TRUE
work_desktop = FALSE
work_linux = TRUE
singularity = TRUE
prj =  "HCC-CBS-173-Hillman-BFerris-NRG-HN003-Vectra"
if (!hpc){
    if (work_desktop){
        if (work_linux){
            base = paste0('/mnt/d/OneDrive - University of Pittsburgh/Internal_Project_Data_2/' ,prj)
        }else{
            base =paste0('D:\\OneDrive - University of Pittsburgh\\Internal_Project_Data_2\\', prj)
        }
    }else{
        base = paste0('/home/brian/data/odrive/Internal_Project_Data_2/',prj)
    }
}else{
    if (singularity){
        base = '/mnt'
    }else{
        base = paste0('/ix/rbao/Projects/', prj)
    }
}
print(base)
inform.data <- file.path(base,'data','inform_data_exports')
results <- file.path(base,'results')
print(results)
fn <- 'cell_density_34_all_inc_tumor_stroma_20_percent_v1.csv'
df <- read.csv(file.path(results,fn))
head(df)


[1] "Loading libraries"
[1] "Library load finished"
Time difference of 0.003573895 secs
[1] "/mnt"
[1] "/mnt/results"


,Sample.Name,study_id,total_roi,n_tumor_segments,n_stroma_segments,PDL1.panCK.tumor_percent,PDL1.panCK.stroma_percent,PDL1.panCK.all_percent,PDL1.panCKneg.CD3pos.tumor_percent,PDL1.panCKneg.CD3pos.stroma_percent,PDL1.panCKneg.CD3pos.all_percent,PDL1.panCKneg.CD3neg.tumor_percent,PDL1.panCKneg.CD3neg.stroma_percent,PDL1.panCKneg.CD3neg.all_percent,CD3.CD8.Tcell.tumor_percent,CD3.CD8.Tcell.stroma_percent,CD3.CD8.Tcell.all_percent,Treg.tumor_percent,Treg.stroma_percent,Treg.all_percent
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,RACTIH_Scan1.qptiff,RACTIH,12,12,12,1.581028,2.4818933,1.932385,0.002949678,0.01845274,0.008996204,0.4277034,2.1220649,1.0885406,0.020647749,0.009226369,0.01619317,0.000000000,0.004613184,0.001799241
2,RACTKE_Scan1.qptiff,RACTKE,285,285,285,68.519584,35.4576429,51.416619,0.049980291,1.07511932,0.580285409,1.1840614,12.9683078,7.2800601,0.055449832,2.010679038,1.06689123,0.092793596,3.922979994,2.074151626
3,RACTLH_Scan2.qptiff,RACTLH,59,59,59,8.792360,3.9117319,6.129775,0.000000000,0.02801540,0.015283563,0.3746256,2.2418837,1.3932923,0.003128397,0.319896799,0.17593870,0.004692596,0.138773968,0.077839544
4,RACTNY_Scan1.qptiff,RACTNY,27,27,27,3.737032,0.8999784,2.049500,0.004234597,0.02019182,0.013726247,0.3917002,0.7427706,0.6005233,0.406521279,1.400447105,0.99772659,0.175735761,0.346145525,0.277098615
5,RACTRF_Scan1.qptiff,RACTRF,10,10,10,13.805149,1.4236082,8.444917,0.058222280,0.14405559,0.095381342,0.2393583,0.7880688,0.4769067,0.226419977,1.559189899,0.80340438,2.083063786,5.363952207,3.503430060
6,RACTRU_Scan1.qptiff,RACTRU,38,38,38,23.019207,8.3016812,11.614162,0.036987134,0.47420639,0.375801253,1.4504240,7.0893089,5.8201625,2.079205305,1.963582790,1.98960600,0.198145359,1.952840251,1.557910260


In [32]:
fns = list.files(inform.data,pattern = '*cell_seg_data.txt')
length(fns)
print(fns[100])

[1] 2029

[1] "RACTKE_Scan1_[12997,52179]_cell_seg_data.txt"


# Load example inForm cell_seg output .txt file:

In [67]:
csd <- read.csv(file.path(inform.data,fns[100]),
               sep = '\t',
               check.names=FALSE) #check.names=FALSE will prevent replacing spaces with periods

# Reformat the cell_seg to contain new unified Phenotype column

In [87]:
unique(csd[,'Tissue Category'])

[1] "Stroma" "Tumor"  "Blank"

In [90]:
p <- c('Phenotype-CK','Phenotype-PDL1','Phenotype-CD3',
       'Phenotype-CD8','Phenotype-PD1','Phenotype-FOXP3')
csd$Phenotype <- ''
for (pheno in p){
    temp = csd[,pheno]
    temp[temp=='other'] = ''
    csd$Phenotype <- paste0(csd$Phenotype,temp)
    }
dim(csd)
# Remove cells with no labeling:

clean.csd <- csd[clean.csd[,'Tissue Category'] != 'Blank',]
clean.csd <- clean.csd %>% filter(Phenotype != '')
dim(clean.csd)

# Remove CK+ cell labeling false alarms:
idx.1 <- clean.csd$Phenotype %like% 'CK+'
print(sum(idx.1))
idx.2 <- !(clean.csd$Phenotype %in% c('CK+','CK+PDL1+','CK+PD1+'))
idx <- idx.1 & idx.2
clean.csd[idx,'Phenotype'] <- substring(clean.csd[idx,'Phenotype'],
                                          first=4,last=nchar(clean.csd$Phenotype[idx]))

# Count
clean.csd %>% count(Phenotype)

[1] 3765  218

[1] 2939  218

[1] 2597


Phenotype,n
<chr>,<int>
CD3+,18
CD3+CD8+,2
CD3+CD8+PD1+,2
CD3+FOXP3+,49
CD3+PD1+,10
CD3+PD1+FOXP3+,2
CD8+,1
CK+,1448
CK+PDL1+,1140


In [89]:
distances <- find_nearest_distance(clean.csd)
csd.dist <- bind_cols(clean.csd, distances)
csd.dist %>% group_by(Phenotype) %>% 
  select(Phenotype, starts_with('Distance to')) %>% 
  summarize_all(~round(mean(.), 1))

Phenotype,Distance to CD3+,Distance to CD3+CD8+,Distance to CD3+CD8+PD1+,Distance to CD3+FOXP3+,Distance to CD3+PD1+,Distance to CD3+PD1+FOXP3+,Distance to CD8+,Distance to CK+,Distance to CK+PDL1+,⋯,Distance to PD1+FOXP3+,Distance to PDL1+,Distance to PDL1+CD3+,Distance to PDL1+CD3+CD8+,Distance to PDL1+CD3+CD8+PD1+,Distance to PDL1+CD3+FOXP3+,Distance to PDL1+CD3+PD1+,Distance to PDL1+FOXP3+,Distance to PDL1+PD1+,Distance to PDL1+PD1+FOXP3+
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CD3+,66.4,412.5,232.5,47.7,94.3,169.8,416.0,34.6,40.9,⋯,248.5,33.8,209.1,542.5,317.2,93.4,300.1,89.7,491.2,325.3
CD3+CD8+,53.0,194.1,614.8,17.3,23.6,496.7,603.0,65.2,95.1,⋯,506.5,32.8,101.7,99.3,137.4,140.1,655.5,31.3,94.3,390.8
CD3+CD8+PD1+,109.1,564.4,452.1,80.3,96.0,116.5,418.9,25.2,19.3,⋯,215.4,25.1,226.8,702.3,411.2,76.7,234.2,124.4,643.8,374.0
CD3+FOXP3+,61.7,287.0,341.0,35.4,72.8,260.1,461.8,43.1,53.5,⋯,310.7,25.7,184.4,407.5,248.6,84.5,409.0,74.6,357.8,335.6
CD3+PD1+,82.2,352.1,335.1,34.1,95.4,274.1,459.4,32.9,52.7,⋯,331.3,22.4,170.0,457.7,291.5,126.4,380.3,89.2,408.2,371.6
CD3+PD1+FOXP3+,12.2,459.2,116.5,16.9,27.5,298.9,385.3,44.0,52.1,⋯,176.9,17.3,197.7,600.8,331.5,29.6,203.1,81.6,540.8,277.8
CD8+,147.3,582.7,209.6,92.5,82.6,247.5,NA,7.3,6.1,⋯,182.4,12.1,506.2,629.3,538.5,178.6,391.1,176.2,582.5,592.9
CK+,132.7,375.9,374.2,104.9,178.6,322.1,529.7,9.6,21.8,⋯,393.5,54.9,240.5,477.5,306.9,177.6,448.9,164.8,436.8,367.8
CK+PDL1+,133.2,396.2,291.3,107.5,160.3,250.8,443.7,19.0,9.6,⋯,308.7,54.5,232.2,507.9,308.3,170.4,378.3,148.7,458.6,337.9


# Examine count_within() command

In [43]:
help(count_within)

count_within {phenoptr},R Documentation
csd,"A data frame with Cell X Position, Cell Y Position and Phenotype columns, such as the result of calling read_cell_seg_data."
"from, to",Selection criteria for the rows and columns. Accepts all formats accepted by select_rows.
radius,The radius or radii to search within.
category,Optional tissue category to restrict both from and to.
dst,"Optional distance matrix corresponding to csd, produced by calling distance_matrix."


In [44]:
# Try count_within() measure CD3+FOXP3+ to CK+

In [53]:
# Find the number of Tregs with a tumor cell 35 microns
outer_margin <- count_within(csd, from='CK+', to='CD3+FOXP3+', 
                             radius=c(10,20,30), category = c('Stroma'))
outer_margin

radius,from_count,to_count,from_with,within_mean
<dbl>,<int>,<int>,<int>,<dbl>
10,109,48,2,0.01834862
20,109,48,4,0.04587156
30,109,48,5,0.05504587


In [50]:
outer_margin

radius,from_count,to_count,from_with,within_mean
<dbl>,<int>,<int>,<int>,<dbl>
10,109,48,2,0.01834862
20,109,48,4,0.04587156
30,109,48,5,0.05504587


Workflow idea: is it possible to subset any cell with < 30um distance to tumor?

ultimately how do I determine N cells within 35um of tumor outer margin / inner margin? create version where tissue category is set as the phenotype? trying this:

In [80]:
temp <- clean.csd
temp$Phenotype <- temp[,'Tissue Category']

distances <- find_nearest_distance(temp)
temp <- bind_cols(temp, distances)
temp %>% group_by(Phenotype) %>% 
  select(Phenotype, starts_with('Distance to')) %>% 
  summarize_all(~round(mean(.), 1))
dist = 100
outer_margin <- (temp[,'Distance to Tumor'] < dist) && (temp[,'Tissue Category'] =='Stroma')
inner_margin <- (temp[,'Distance to Stroma'] < dist ) && (temp[,'Tissue Category'] =='Tumor')
print(sum(outer_margin))
print(sum(inner_margin))
# outer_margin <- count_within(temp, from='Stroma', to='Tumor', 
#                              radius=c(35))
# outer_margin

Phenotype,Distance to Blank,Distance to Stroma,Distance to Tumor
<chr>,<dbl>,<dbl>,<dbl>
Blank,NA,14.4,56.2
Stroma,347.9,12.1,24.2
Tumor,351.1,34.8,8.7


[1] 0
[1] 1


In [79]:
sum(temp[,'Tissue Category'] =='Stroma')

[1] 585

In [78]:
sum(temp[,'Distance to Tumor'] < 35 )

[1] 2791

# Loop through files and measure how many cells in the margins

In [97]:

dim(df)
head(df)

[1] 2029    2

,inner_n,outer_n
,<lgl>,<lgl>
"RACTIH_Scan1_[15509,42225]_cell_seg_data.txt",NA,NA
"RACTIH_Scan1_[15509,42918]_cell_seg_data.txt",NA,NA
"RACTIH_Scan1_[15509,43611]_cell_seg_data.txt",NA,NA
"RACTIH_Scan1_[15509,44304]_cell_seg_data.txt",NA,NA
"RACTIH_Scan1_[16434,42225]_cell_seg_data.txt",NA,NA
"RACTIH_Scan1_[16434,42918]_cell_seg_data.txt",NA,NA


In [ ]:
p <- c('Phenotype-CK','Phenotype-PDL1','Phenotype-CD3',
   'Phenotype-CD8','Phenotype-PD1','Phenotype-FOXP3')
df <- data.frame(matrix(NA, nrow = length(fns), ncol = 2),row.names=fns)
colnames(df) <- c('inner_n','outer_n')
for (fn in fns){
    csd <- read.csv(file.path(inform.data,fn),
               sep = '\t',
               check.names=FALSE) #check.names=FALSE will prevent replacing spaces with periods\
    csd$Phenotype <- ''
    for (pheno in p){
        temp = csd[,pheno]
        temp[temp=='other'] = ''
        csd$Phenotype <- paste0(csd$Phenotype,temp)
        }
    # dim(csd)
    # Remove cells with no labeling:

    clean.csd <- csd[clean.csd[,'Tissue Category'] != 'Blank',]
    clean.csd <- clean.csd %>% filter(Phenotype != '')
    # dim(clean.csd)

    # Remove CK+ cell labeling false alarms:
    idx.1 <- clean.csd$Phenotype %like% 'CK+'
    # print(sum(idx.1))
    idx.2 <- !(clean.csd$Phenotype %in% c('CK+','CK+PDL1+','CK+PD1+'))
    idx <- idx.1 & idx.2
    clean.csd[idx,'Phenotype'] <- substring(clean.csd[idx,'Phenotype'],
                                              first=4,last=nchar(clean.csd$Phenotype[idx]))
    
    temp <- clean.csd
    temp$Phenotype <- temp[,'Tissue Category']

    distances <- find_nearest_distance(temp)
    temp <- bind_cols(temp, distances)
    temp %>% group_by(Phenotype) %>% 
      select(Phenotype, starts_with('Distance to')) %>% 
      summarize_all(~round(mean(.), 1))
    dist = 100
    outer_margin <- (temp[,'Distance to Tumor'] < dist) && (temp[,'Tissue Category'] =='Stroma')
    inner_margin <- (temp[,'Distance to Stroma'] < dist ) && (temp[,'Tissue Category'] =='Tumor')
    df[fn,'inner_n'] <- sum(inner_margin)
    df[fn,'outer_n'] <- sum(outer_margin)
    }

In [ ]:
mean(df)

In [ ]:
help(mean)